# Script to diagnose the nature of the eigenvalues from the RSW $\eta$ modal decompostion
### twnh Sep, Oct, Nov '21

#### -1. Housekeeping

In [1]:
using Gridap, GridapGmsh, GridapMakie
#using GLMakie
#GLMakie.inline!(true)    # Makes sure figures appear in notebook inline.
using CairoMakie
using LinearAlgebra
using NonlinearEigenproblems

using Printf
using Glob
using LaTeXStrings

include("RSW_ModelFunctions.jl")     # Function definitions
checkEigenFlag = true ;       # Check if eigensolution satisfies original nonlinear eigen problem
trimThreshold = 1.0e-6 ;       # Threshold to trim small eigenvalues. Check figures to make sure this is OK.

In [2]:
#domainName = "square_384nodes_uneven.msh"
#domainName = "square_803nodes_uneven.msh"
#domainName = "square_145nodes.msh"
#domainName = "square_514nodes.msh"
domainName = "circle_268nodes.msh"
#domainName = "circle_992nodes.msh"

# The non-dimensionalization sets the longest lengthscale of the non-dimensional domain equal to one.
 U, V, U∞, V∞, Ω, dΩ, dΓ, t_Γ, n_nodes = DefineProblem(domainName)

# Physical parameters: Avoid integer multiples of these parameters (they can throw off the polynomial eigensolver)
#F = (2.0 .^ range(-1.001, 11.001, length = 7))
#Fs = (2.0 .^ range(-1.001, 12.001, step = 1.0))
Fs = (2.0 .^ range(-1.001, 12.001, step = 0.125/4))
#Fs = (2.0 .^ range( 2.001, 5.001, step = 0.125/8))
 f = 1.0e-4     # s^{-1}
 g = 9.81     # ms^{-2}
 L = 512.0e3    # m 
 Lᵣ = sqrt.(L^2 ./ Fs)
 H = ((f * Lᵣ) .^ 2) ./ g
 timescale = L ./ sqrt.(g .* H)
@printf("Coriolis parameter                 = [%8.2e] s^{-1}\n", f)
@printf("Gravitational acceleration         = [%8.2f] ms^{-2}\n", g)
@printf("Layer thickness                    = [%8.1f]--[%8.1f] m\n", maximum(H), minimum(H))
@printf("Domain lengthscale                 = [%8.2f] km\n", L / 1e3)
@printf("Deformation radius                 = [%8.2f]--[%8.2f] km\n", maximum(Lᵣ) / 1e3, minimum(Lᵣ) / 1e3)
@printf("Characteristic timescale           = [%8.2f]--[%8.2f] days.\n", maximum(timescale) / 86400, minimum(timescale) / 86400)
@printf("\nInverse Froude number            F = [%8.2f]--[%8.2f] (%d of them)\n", minimum(Fs), maximum(Fs), size(Fs,1))
@printf("Non-dimensional deformation rad.   = [%8.2f]--[%8.2f]\n", maximum(1 ./ sqrt.(Fs)), minimum(1 ./ sqrt.(Fs)))
@printf("\nMesh has                           = [%6.0d] nodes.\n", n_nodes)

Reading Gmsh discrete model mesh [circle_268nodes.msh]
Info    : Reading 'circle_268nodes.msh'...
Info    : 3 entities
Info    : 268 nodes
Info    : 534 elements
Info    : Done reading 'circle_268nodes.msh'
Coriolis parameter                 = [1.00e-04] s^{-1}
Gravitational acceleration         = [    9.81] ms^{-2}
Layer thickness                    = [   534.8]--[     0.1] m
Domain lengthscale                 = [  512.00] km
Deformation radius                 = [  724.33]--[    8.00] km
Characteristic timescale           = [    7.40]--[    0.08] days.

Inverse Froude number            F = [    0.50]--[ 4093.16] (417 of them)
Non-dimensional deformation rad.   = [    1.41]--[    0.02]

Mesh has                           = [   268] nodes.


In [3]:
# Loop over different F values calculating the eigenmodes
 NF = size(Fs, 1)
 nev = 3 * n_nodes
ωs = Array{ComplexF64}(undef,nev,NF)
ηs = Array{CellField}(undef,nev,NF)
for ff = 1:NF
    F = Fs[ff]
    ω, _, η, _ = solveModesProblem(F)
    ωs[1:size(ω,1), ff] = ω
    ηs[1:size(η,1), ff] = η
end # ff


Solving modal problem for (ω,η) eigenpairs with F = [  0.4997]:
Done in [ 27.89]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [1.23e-19 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   789], ω = [-85.811 +  0.000im], max residual = [1.182e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  0.5106]:
Done in [  4.82]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [1.39e-19 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   788], ω = [-85.725 +  0.000im], max residual = [4.317e-10]

Solving modal problem for (ω,η) eigenpairs with F = [  0.5218]:
Done in [  4.64]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue r

Number of eigenvalues              = [   804]
Eigenvalue range                   = [7.67e-19 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   782], ω = [-84.457 +  0.000im], max residual = [1.548e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  0.8775]:
Done in [  4.94]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [3.58e-19 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   796], ω = [-88.288 + -0.000im], max residual = [1.069e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  0.8967]:
Done in [  2.13]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [2.12e-19 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = 

Maximum residual for eigenvalue nn = [   800], ω = [-89.705 + -0.000im], max residual = [1.591e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  1.5081]:
Done in [  3.52]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [4.81e-20 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   765], ω = [-78.618 +  0.000im], max residual = [1.359e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  1.5411]:
Done in [  1.86]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [2.44e-20 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   787], ω = [-85.731 +  0.000im], max residual = [1.137e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  1.5749]:
Done in [  4.61]s.

Matrix proble

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [1.55e-19 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   776], ω = [-82.581 + -0.000im], max residual = [1.087e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  2.6486]:
Done in [  2.10]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [2.48e-19 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   785], ω = [85.588 +  0.000im], max residual = [1.477e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  2.7066]:
Done in [  1.94]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [3.42e-21 -- 1.01e+02]
Size of eigenvector matrix         = [

Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   790], ω = [-85.834 +  0.000im], max residual = [1.101e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  4.5520]:
Done in [  2.12]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [4.87e-20 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   774], ω = [-82.508 + -0.000im], max residual = [1.344e-09]

Solving modal problem for (ω,η) eigenpairs with F = [  4.6517]:
Done in [  2.02]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [1.91e-19 -- 1.01e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   781], ω = [-84.480 + -0.000im], max residual = [1.263e-09]

Solving modal problem for (ω,η) eigenpairs

...

Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   656], ω = [-67.627 +  0.000im], max residual = [1.310e-09]

Solving modal problem for (ω,η) eigenpairs with F = [377.8055]:
Done in [  1.98]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [3.68e-20 -- 1.03e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   656], ω = [-67.689 + -0.000im], max residual = [1.197e-09]

Solving modal problem for (ω,η) eigenpairs with F = [386.0784]:
Done in [  1.90]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [9.66e-20 -- 1.03e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   656], ω = [-67.752 + -0.000im], max residual = [9.300e-10]

Solving modal problem for (ω,η) eigenpairs

Done in [  4.31]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [6.93e-20 -- 1.04e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   351], ω = [-35.617 + -0.000im], max residual = [5.448e-09]

Solving modal problem for (ω,η) eigenpairs with F = [663.5218]:
Done in [  1.89]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [1.71e-19 -- 1.04e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   674], ω = [71.253 + -0.000im], max residual = [6.778e-10]

Solving modal problem for (ω,η) eigenpairs with F = [678.0510]:
Done in [  2.03]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [6.26e-21 -- 1.04e+02]
Size of eigenvecto

Maximum residual for eigenvalue nn = [   727], ω = [-79.899 + -0.000im], max residual = [2.921e-10]

Solving modal problem for (ω,η) eigenpairs with F = [1140.3413]:
Done in [  4.36]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [1.44e-20 -- 1.07e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   757], ω = [-83.843 +  0.000im], max residual = [3.096e-10]

Solving modal problem for (ω,η) eigenpairs with F = [1165.3115]:
Done in [  4.27]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [3.82e-20 -- 1.07e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   731], ω = [-80.805 + -0.000im], max residual = [3.460e-10]

Solving modal problem for (ω,η) eigenpairs with F = [1190.8285]:
Done in [  2.14]s.

Matrix pro

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [6.32e-20 -- 1.10e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   685], ω = [-80.648 + -0.000im], max residual = [6.723e-10]

Solving modal problem for (ω,η) eigenpairs with F = [2002.7269]:
Done in [  4.39]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [2.71e-20 -- 1.11e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   647], ω = [-78.173 + -0.000im], max residual = [3.723e-10]

Solving modal problem for (ω,η) eigenpairs with F = [2046.5809]:
Done in [  3.89]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [1.84e-20 -- 1.11e+02]
Size of eigenvector matrix         

Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   655], ω = [87.200 +  0.000im], max residual = [7.007e-10]

Solving modal problem for (ω,η) eigenpairs with F = [3441.9251]:
Done in [  2.01]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [6.82e-20 -- 1.17e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   586], ω = [-83.372 +  0.000im], max residual = [7.755e-10]

Solving modal problem for (ω,η) eigenpairs with F = [3517.2935]:
Done in [  3.56]s.

Matrix problem size                = [   268 x    268]
Number of eigenvalues              = [   804]
Eigenvalue range                   = [3.09e-19 -- 1.17e+02]
Size of eigenvector matrix         = [   804 x      1]
Maximum residual for eigenvalue nn = [   673], ω = [-89.117 + -0.000im], max residual = [5.308e-10]

Solving modal problem for (ω,η) eigenpair

In [4]:
# Plot frequencies
fig = Figure(resolution = (1600,1600))
ax1 = Axis(fig[1, 1], title = "Eigenvalues vs F", xlabel = "F", ylabel = "eigenvalues")
lines!(Fs, sqrt.(Fs),linewidth = 4.0)
lines!(Fs, sqrt.(Fs) .* 0.025,linewidth = 4.0)
for ff = 1:NF
    inds = findall(x -> x > 1.0e-8, real(ωs[:,ff]))
    scatter!(fig[1, 1], Fs[ff] .* ones(size(inds,1)), real(ωs[inds,ff]), markersize = 2, color = :black)
end # ff
ax1.xscale = log2
ax1.yscale = log10
xlims!(ax1,minimum(Fs),maximum(Fs))
ylims!(ax1,1e-2,1e3)
#display(fig)
save("output.pdf",fig)

CairoScreen{Cairo.CairoSurfaceIOStream{UInt32}} with surface:
Cairo.CairoSurfaceIOStream{UInt32}(Ptr{Nothing} @0x00007f9b7c94d710, 1200.0, 1200.0, IOContext(IOStream(<file /Users/thaine1/Library/CloudStorage/OneDrive-JohnsHopkins/Documents/Julia/gridap/RSW_model/output.pdf>)))


In [5]:
# Plot modes
#ffs = 1:2:NF  # F numbers (across columns)
ffs = 1:64:NF  # F numbers (across columns)
mms = 1:8     # modes (down rows)

fig2 = Figure(resolution = (1600, 2000), font =:sans)
for (jj,ff) in zip(1:size(ffs,1),ffs)
  for (ii,mm) in zip(1:size(mms,1),mms)
      thisω = round(real(ωs[mm,ff]),digits=1)
      thisF = round(Fs[ff],digits=1)
      thisη = abs(ηs[mm,ff])
      tit_txt1 = L"$\Re ( \eta_{%$(mm)})$ $\omega = %$(thisω)$, $F = %$(thisF)$"
      ax2 = Axis(fig2[ii,jj],xlabel=L"x",ylabel=L"y",title=tit_txt1)        
      ax2.aspect = DataAspect()
      hidedecorations!(ax2)
      plot!(Ω,thisη)
  end # for
end # for
#display(fig2)
save("output_modes.pdf",fig2)

CairoScreen{Cairo.CairoSurfaceIOStream{UInt32}} with surface:
Cairo.CairoSurfaceIOStream{UInt32}(Ptr{Nothing} @0x00007f9c29c333d0, 1200.0, 1500.0, IOContext(IOStream(<file /Users/thaine1/Library/CloudStorage/OneDrive-JohnsHopkins/Documents/Julia/gridap/RSW_model/output_modes.pdf>)))


In [6]:
ff = 12
this_F = round(Fs[ff],digits=1)
global inds = findall(x -> x > 1.0e-12, real(ωs[:,ff]))
ω = ωs[inds,ff]

fig3 = Figure(resolution = (1200,600))
Axis(fig3[1, 1], title = "Eigenvalue F = $(this_F)", ylabel = "eigenvalues", xlabel = "eigenvalue index")
tmp = abs.(real(ω))
scatter!(fig3[1, 1], tmp, markersize = 4, color = :black)

Axis(fig3[1, 2], title = "Eigenvalue differences F = $(this_F)", ylabel = "differences", xlabel = "eigenvalue index")
tmp = diff(abs.(real(ω)))
scatter!(fig3[1, 2], tmp, markersize = 4, color = :black)

ratiom1 = (abs.(ω[2:end]) ./ abs.(ω[1])) .- 1.0
Axis(fig3[1, 3], title = "Eigenvalue ratios  F = $(this_F)", ylabel = "ratios", xlabel = "eigenvalue index")
scatter!(fig3[1, 3], real(ratiom1), markersize = 4, color = :black)
ylims!(1, 2000)
#display(fig3)
#ratiom1
save("output3.pdf",fig3)

CairoScreen{Cairo.CairoSurfaceIOStream{UInt32}} with surface:
Cairo.CairoSurfaceIOStream{UInt32}(Ptr{Nothing} @0x00007f9b7cac16f0, 900.0, 450.0, IOContext(IOStream(<file /Users/thaine1/Library/CloudStorage/OneDrive-JohnsHopkins/Documents/Julia/gridap/RSW_model/output3.pdf>)))
